# Examples of the available subclass of InverseProblem

We import the necessary packages and create a function for printing the
names and shapes of the created training sets.

In [6]:
import numpy as np
import invprob

def print_shapes(name_length=10, **kwargs):
    '''Prints the shapes for given keyword-array pairs.'''
    length = '%-' + str(name_length) + 's'
    print(length % 'var', 'shape')
    for name, data in kwargs.items():
        print(length % name, str(data.shape))

## The Gaussian Inverse Problem class
There are three possibilities for creating a Gaussian Inverse Problem.

#### 1. User-specified forward operator

You can provide your own forward operator as an ```ndarray``` of shape ```(dim_Y, dim_X)```.


In [7]:
A = np.arange(12).reshape((4,3))

gaussian_custom = invprob.GaussianInverseProblem(forward_operator=A)
X, Y, Z = gaussian_custom.create_samples(n_samples=10)
print_shapes(X=X, Y=Y, Z=Z, A=gaussian_custom.A)

var        shape
X          (10, 3)
Y          (10, 4)
Z          (10, 4)
A          (4, 3)


#### 2. Identity forward operator

You can choose the identity as a forward operator.

In [8]:
N = 4
gaussian_identity = invprob.GaussianInverseProblem(forward_operator='identity', 
                                             dim_X=N)
X, Y, Z = gaussian_identity.create_samples(n_samples=10)
print_shapes(X=X, Y=Y, Z=Z, A=gaussian_identity.A)

var        shape
X          (10, 4)
Y          (10, 4)
Z          (10, 4)
A          (4, 4)


#### 3. Forward operator with independent standard normal entries

You can let the entries of A be realizations of independent standard normal Gaussian random variables.

In [9]:
N = 8
M = 16
gaussian_random = invprob.GaussianInverseProblem(forward_operator='random', 
                                             dim_X=N, dim_Y=M)
X, Y, Z = gaussian_random.create_samples(n_samples=10)
print_shapes(X=X, Y=Y, Z=Z, A=gaussian_random.A)

var        shape
X          (10, 8)
Y          (10, 16)
Z          (10, 16)
A          (16, 8)


## The Sampled Inverse Problem class

In the ```SampledInverseProblem``` class the prior is only observed through
samples from some database.  In this example, as a database we use the [Fashion MNIST image data](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data) 
which can be loaded from TensorFlow.


In [10]:
from tensorflow import keras

def rescaled_fashion_mnist():
    '''Loads and rescales the fashion_mnist training and test images.
    
    Returns:
        X_train: ndarray of shape (60000, 28**2)
            The 60000 training images in vectorized form. 
        X_test: ndarray of shape (10000, 28**2)
            The 10000 test images in vectorized form.
    '''    
    fashion_mnist = keras.datasets.fashion_mnist
    (train_img, train_labels), (test_img, test_labels) = (
        fashion_mnist.load_data())

    train_img = train_img / 255 
    test_img = test_img / 255

    X_train = np.reshape(
        train_img, 
        (train_img.shape[0], train_img.shape[1]*train_img.shape[2])
        )
    X_test = np.reshape(
        test_img, 
        (test_img.shape[0], test_img.shape[1]*test_img.shape[2])
        )
    return X_train, X_test

denoising_fashion_mnist = invprob.SampledInverseProblem(
    database=rescaled_fashion_mnist,
    noiselevel=0.3
    )
            
X_test, Y_test, Z_test = denoising_fashion_mnist.create_samples(n_samples=300, 
                                            sample_type='test')
X_train, Y_train, Z_train = denoising_fashion_mnist.create_samples(
    n_samples=500, 
    sample_type='train'
    )
print_shapes(X_test=X_test, Y_test=Y_test, Z_test=Z_test, X_train=X_train, 
             Y_train=Y_train, Z_train=Z_train, A=denoising_fashion_mnist.A)    

var        shape
X_test     (300, 784)
Y_test     (300, 784)
Z_test     (300, 784)
X_train    (500, 784)
Y_train    (500, 784)
Z_train    (500, 784)
A          (784, 784)
